In [1]:
import pandas as pd
import numpy as np
import re
import csv
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

np.random.seed(500)

In [2]:
df_prescriptions = pd.read_csv('C:/Users/harim/OneDrive/Documents/GitHub/dl4h_final_project_1/data/processed/ndc_codes_extracted.csv', dtype=str)

df_prescriptions.head()

,Unnamed: 0,NDC,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC
0,0,00469061711,Tacrolimus,Tacrolimus,Tacrolimus
1,1,00056017275,Warfarin,Warfarin,Warfarin
2,2,00338055002,Heparin Sodium,UNKNOWN,UNKNOWN
3,4,00054829725,Furosemide,Furosemide,Furosemide
4,5,00056016975,Warfarin,Warfarin,Warfarin


In [3]:

# ndc_codes = set()

# for index, record in df_prescriptions.iterrows():
#     ndc_codes.add(record['NDC'].strip())
        
# ndc_codes = list(ndc_codes)

# print(len(ndc_codes))

In [4]:
df = pd.read_csv('../data/processed/NOTE_EVENTS_WITH_DRUGS.csv', dtype=str)

# df = df.head(1000)

In [5]:
df['DRUGS_PRESENT_SPLIT'] = df['DRUGS_PRESENT'].str.split(pat=',') #.apply(lambda x: x.split(','))

df = df[df['DRUGS_PRESENT_SPLIT'].notnull()]

print("Count After Null Filter:", df.count())

Count After Null Filter: ROW_ID                 48589
SUBJECT_ID             48589
HADM_ID                48589
CHARTDATE              48589
CHARTTIME                  0
STORETIME                  0
CATEGORY               48589
DESCRIPTION            48589
CGID                       0
ISERROR                    0
TEXT                   48589
CLEAN_TEXT             48589
DRUGS_PRESENT          48589
DRUGS_PRESENT_SPLIT    48589
dtype: int64


In [6]:

free_text = df['CLEAN_TEXT']
label_codes = df['DRUGS_PRESENT_SPLIT']

### Text Vectorization

In [ ]:

text_vectorizer = TfidfVectorizer(ngram_range=(3, 3),
                             analyzer='char')
X = text_vectorizer.fit_transform(free_text)

In [ ]:
print(X.shape, text_vectorizer.get_feature_names())
print("....", X[0,0])

### Label Encoding

In [ ]:
Y = np.array(label_codes)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
    
lb = MultiLabelBinarizer()
Y = lb.fit_transform(Y)

# Y = Y.T

In [ ]:
print("Labels found in first example: ", Y[0].sum())

print(Y[0])

In [ ]:
print("X: ", X.shape, " Y: ", Y.shape)

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109)

In [ ]:
# from sklearn import svm
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression

# #Create a svm Classifier
# model = DecisionTreeClassifier(max_depth=5) # Linear Kernel

# #Train the model using the training sets
# model.fit(X_train, y_train)

# #Predict the response for test dataset
# y_pred = model.predict(X_test)

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=500, max_iter=100)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
print(y_pred[1])

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred, average='micro'))

In [ ]:
import pickle
with open('C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\vectorizer.pk', 'wb') as fin:
    pickle.dump(text_vectorizer, fin)

In [ ]:
output = open('C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\label_encoder.pkl', 'wb')
pickle.dump(lb, output)
output.close()

In [ ]:
from joblib import dump, load

dump(model, 'C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\model.joblib') 